# Using ShEx with PyShExs
## 1. Local file, all subjects

In [15]:
import rdflib
from pyshex.shex_evaluator import ShExEvaluator
from pyshex.user_agent import SlurpyGraphWithAgent
from pyshex.utils.sparql_query import SPARQLQuery

### Open local file

In [29]:
var = open('aop.ttl', 'r').read()

### Find all subjects

In [30]:
g = rdflib.Graph()
g.parse('aop.ttl', format="ttl") #adapt format if necessary
nodes = set()
for subject, predicate, object in g.triples((None, None, None)):
    nodes.add(subject)
print(nodes)

{rdflib.term.URIRef('http://identifiers.org/aop.relationships/597'), rdflib.term.URIRef('http://identifiers.org/aop.events/593'), rdflib.term.URIRef('http://identifiers.org/aop.events/585')}


### Define Shape Expression to test on the subjects (nodes)

In [31]:
shex = """
PREFIX aop.relationships: <http://identifiers.org/aop.relationships/> 
PREFIX aop.events: <http://identifiers.org/aop.events/> 
PREFIX aopo: <http://aopkb.org/aop_ontology#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 

start = @<ker>
<ker> {
  aopo:has_upstream_key_event     @<ke> ;
  aopo:has_downstream_key_event   @<ke>
}

<ke> {
   a [ aopo:KeyEvent ]
}"""

### Execute

In [32]:
result = ShExEvaluator(var, shex, nodes).evaluate()

for r in result:
    print(f"{r.focus}: ", end="")
    if not r.result:
        print(f"FAIL: {r.reason}")
    else:
        print("PASS")

http://identifiers.org/aop.relationships/597: PASS
http://identifiers.org/aop.events/593: FAIL:   Testing aop.events:593 against shape ker
       No matching triples found for predicate aopo:has_upstream_key_event
http://identifiers.org/aop.events/585: FAIL:   Testing aop.events:585 against shape ker
       No matching triples found for predicate aopo:has_upstream_key_event


## 2. SPARQL endpoint, selected subjects

In [10]:
from pyshex.shex_evaluator import ShExEvaluator
from pyshex.user_agent import SlurpyGraphWithAgent
from pyshex.utils.sparql_query import SPARQLQuery

### Define the SPARQL endpoint URL

In [11]:
endpoint = 'http://aopwiki-rdf.prod.openrisknet.org/sparql/'

### Define the SPARQL query to extract subjects to test on with the ShExs

In [12]:
sparql = """
PREFIX aop.relationships: <http://identifiers.org/aop.relationships/> 
PREFIX aop.events: <http://identifiers.org/aop.events/> 
PREFIX aopo: <http://aopkb.org/aop_ontology#> 
SELECT DISTINCT ?item WHERE {
  ?item rdf:type aopo:KeyEventRelationship
}
LIMIT 20
"""

### Define Shape Expression to test on the subjects (nodes)

In [13]:
shex = """
PREFIX aop.relationships: <http://identifiers.org/aop.relationships/> 
PREFIX aop.events: <http://identifiers.org/aop.events/> 
PREFIX aopo: <http://aopkb.org/aop_ontology#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 

start = @<ker>
<ker> {
  aopo:has_upstream_key_event     @<ke> ;
  aopo:has_downstream_key_event   @<ke>
}

<ke> {
   a [ aopo:KeyEvent ]
}"""

### Execute

In [14]:
result = ShExEvaluator(SlurpyGraphWithAgent(endpoint),
                       shex,
                       SPARQLQuery(endpoint, sparql).focus_nodes()).evaluate()
for r in result:
    print(f"{r.focus}: ", end="")
    if not r.result:
        print(f"FAIL: {r.reason}")
    else:
        print("PASS")

http://identifiers.org/aop.relationships/1017: PASS
http://identifiers.org/aop.relationships/1023: PASS
http://identifiers.org/aop.relationships/1024: PASS
http://identifiers.org/aop.relationships/1026: PASS
http://identifiers.org/aop.relationships/1027: PASS
http://identifiers.org/aop.relationships/1034: PASS
http://identifiers.org/aop.relationships/1035: PASS
http://identifiers.org/aop.relationships/1036: PASS
http://identifiers.org/aop.relationships/1037: PASS
http://identifiers.org/aop.relationships/1039: PASS
http://identifiers.org/aop.relationships/1041: PASS
http://identifiers.org/aop.relationships/1042: PASS
http://identifiers.org/aop.relationships/1044: PASS
http://identifiers.org/aop.relationships/1069: PASS
http://identifiers.org/aop.relationships/1070: PASS
http://identifiers.org/aop.relationships/1143: PASS
http://identifiers.org/aop.relationships/1268: PASS
http://identifiers.org/aop.relationships/1269: PASS
http://identifiers.org/aop.relationships/1271: PASS
http://ident